# Bitcoin price forecasting with PySpark - Data crawling
## Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



## Global constants, dependencies, libraries and tools

In [ ]:
# Main constants
GDRIVE_DIR = "/content/drive"

In [ ]:
# Datasets dir
GDRIVE_DATASET_RAW_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/raw"

# Datasets name
DATASET_NAME = "bitcoin_blockchain_data_30min"

# Datasets path
GDRIVE_DATASET_RAW = GDRIVE_DATASET_RAW_DIR + "/" + DATASET_NAME + ".parquet"

In [ ]:
# Useful imports
import pandas as pd
import functools

from google.colab import drive

from datetime import date

In [ ]:
# Point Colaboratory to our Google Drive
from google.colab import drive

# Define GDrive paths
drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


In [ ]:
# GDRIVE_UTILITIES_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/utilities"

# import sys
# sys.path.append(GDRIVE_UTILITIES_DIR)

# from imports import *

# imports.USE_PYSPARK = False

##  Define metrics and parameters

In this section we are going to define the parameters used to collect the data and the metrics used. I will consider Bitcoin data for 10 years, starting from 2012-01-01 through 2022-12-31.

Note that since the Blockchain.com API allows retreiving data with a maximum timespan equal to 6 years, I manually computed the continue date so that I could make a second API call to get the remaining data.

Regarding the metrics, I chose the ones that seemed most relevant to me, containing both price statistics but also technical features of Bitcoin's blockchain.

In [ ]:
# Get the current date
current_date = date.today()

# Print the current date
print(current_date)

2023-08-25


In [ ]:
# Define the parameters
timespan = "6years" # Duration of the data (because the Max timespan == 6years)
start_date = "2016-01-01"
continue_date = "2021-12-31" # The continue date (manually calculate the continue_date)
end_date = str(current_date)

# Metrics considered
metrics = [
          ##Currency Statistics##
            "market-price", # Market Price: The average USD market price across major bitcoin exchanges.
            "trade-volume", #Exchange Trade Volume (USD): The total USD value of trading volume on major bitcoin exchanges.

          ##Block Details##
            "blocks-size", # Blockchain Size (MB): The total size of the blockchain minus database indexes in megabytes.
            "avg-block-size", # Average Block Size (MB): The average block size over the past 24 hours in megabytes.
            "n-transactions-total", # Total Number of Transactions: The total number of transactions on the blockchain.
            "n-transactions-per-block", # Average Transactions Per Block: The average number of transactions per block over the past 24 hours.

          ##Mining Information##
            "hash-rate", # Total Hash Rate (TH/s): The estimated number of terahashes per second the bitcoin network is performing in the last 24 hours.
            "difficulty", # Network Difficulty (T): A relative measure of how difficult it is to mine a new block for the blockchain.
            "miners-revenue", # Miners Revenue (USD): Total value of coinbase block rewards and transaction fees paid to miners.
            "transaction-fees-usd", # Total Transaction Fees (USD): The total USD value of all transaction fees paid to miners. This does not include coinbase block rewards.

          ##Network Activity##
            "n-unique-addresses", # The total number of unique addresses used on the blockchain.
            "n-transactions", # Confirmed Transactions Per Day: The total number of confirmed transactions per day.
            "estimated-transaction-volume-usd" # Estimated Transaction Value (USD): The total estimated value in USD of transactions on the blockchain. This does not include coins returned as change.
]

## Retreiving data

In this section we are going to make the call to the Blockchain.com API to retrieve the data.

In [ ]:
def data_crawler(timespan, metrics, start_date, continue_date, end_date):
    # API Info
    url1 = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={start_date}&format=csv'
    url2 = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={continue_date}&format=csv'

    # Obtain Data
    data1 = pd.read_csv(url1,names=['timestamp',metrics])
    data2 = pd.read_csv(url2,names=['timestamp',metrics])

    # Concat by rows
    all_data = pd.concat([data1,data2])

    # Transform "timestamp" to datetime type
    all_data['timestamp'] = pd.to_datetime(all_data["timestamp"])

    # Keep the same end date with Bitcoin data
    all_data = all_data[(all_data['timestamp'] < end_date)]

    return all_data

In [ ]:
# Merge the data
merge = functools.partial(pd.merge, on='timestamp')

# Gain Blockchain Data from Blockchain.com API
df1 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, continue_date, end_date) for metric in metrics])
df1

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07
2,2016-01-03,432.76,1.287046e+06,54754.876205,0.554656,101427625,983.503448,7.694240e+05,1.038803e+11,1.720316e+06,11009.108568,335666.0,142608.0,6.764693e+07
3,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07
4,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2781,2023-08-20,26090.70,5.461775e+07,504843.458513,1.721834,881712238,3780.623457,4.219095e+08,5.239118e+13,2.719019e+07,395205.179498,691316.0,612461.0,1.116745e+09
2782,2023-08-21,26189.99,3.576662e+07,505121.891014,1.633983,882324258,3333.781818,4.297226e+08,5.239118e+13,2.734033e+07,432459.701750,739287.0,550074.0,2.438814e+09
2783,2023-08-22,26123.41,8.358734e+07,505391.626186,1.728887,882874631,2943.055944,3.809369e+08,5.358841e+13,2.404616e+07,492337.168420,633631.0,420857.0,2.805355e+09
2784,2023-08-23,26036.91,9.725932e+07,505638.787271,1.687615,883294423,2942.408451,3.926239e+08,5.562144e+13,2.371891e+07,506433.385402,652036.0,417822.0,4.105798e+09


In [ ]:
# Check duplicated rows
len(df1['timestamp'].unique())

2786

Due to a problem with the Blockchain.com API, I was forced to make an additional call to retrieve capitalization and total circulating data that will be added to the currency statistics to get a single dataset.

In [ ]:
# Retrieving market capitalization and total circulating data
metrics = [
  "total-bitcoins", # Total Circulating Bitcoin: The total number of mined bitcoin that are currently circulating on the network.
  "market-cap", # Market Capitalization (USD): The total USD value of bitcoin in circulation.
  ]

merge = functools.partial(pd.merge, on='timestamp')
df2 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, continue_date, end_date) for metric in metrics])
df2

,timestamp,total-bitcoins,market-cap
0,2016-01-01 00:03:21,15029575.00,6.474140e+09
1,2016-01-02 15:54:15,15035125.00,6.499685e+09
2,2016-01-04 02:26:37,15040650.00,6.458253e+09
3,2016-01-05 09:50:06,15046150.00,6.489359e+09
4,2016-01-06 21:03:44,15051750.00,6.472554e+09
...,...,...,...
2992,2023-08-23 04:18:26,19465368.75,5.060607e+11
2993,2023-08-23 12:08:20,19465737.50,5.052332e+11
2994,2023-08-23 23:36:50,19466106.25,5.159686e+11
2995,2023-08-24 08:54:14,19466475.00,5.151803e+11


In [ ]:
# Check duplicated rows
len(df2['timestamp'].unique())

2997

In [ ]:
# Wipe off the timestamp's h:m:s.
df2['timestamp'] = pd.to_datetime(df2["timestamp"]).dt.normalize()

# Drop the duplicates in column "timestamp", keep the last value
df2.drop_duplicates(subset="timestamp", keep="last", inplace=True)

In [ ]:
df2

,timestamp,total-bitcoins,market-cap
0,2016-01-01,15029575.00,6.474140e+09
1,2016-01-02,15035125.00,6.499685e+09
2,2016-01-04,15040650.00,6.458253e+09
3,2016-01-05,15046150.00,6.489359e+09
4,2016-01-06,15051750.00,6.472554e+09
...,...,...,...
2986,2023-08-20,19463150.00,5.090198e+11
2989,2023-08-21,19464256.25,5.101582e+11
2991,2023-08-22,19465000.00,5.048637e+11
2994,2023-08-23,19466106.25,5.159686e+11


In [ ]:
# Check duplicated rows
len(df2['timestamp'].unique())

2106

In [ ]:
# Add the market capitalization and total circulating data
all_data = pd.merge(df1, df2, how="inner", on='timestamp')
all_data = all_data.interpolate(method='ffill')
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07,15029575.00,6.474140e+09
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07,15035125.00,6.499685e+09
2,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07,15040650.00,6.458253e+09
3,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08,15046150.00,6.489359e+09
4,2016-01-06,432.43,1.677504e+06,55036.950659,0.662380,101934175,1220.496454,7.022931e+05,1.038803e+11,1.702648e+06,14113.278937,312004.0,172090.0,1.007001e+08,15051750.00,6.472554e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2097,2023-08-20,26090.70,5.461775e+07,504843.458513,1.721834,881712238,3780.623457,4.219095e+08,5.239118e+13,2.719019e+07,395205.179498,691316.0,612461.0,1.116745e+09,19463150.00,5.090198e+11
2098,2023-08-21,26189.99,3.576662e+07,505121.891014,1.633983,882324258,3333.781818,4.297226e+08,5.239118e+13,2.734033e+07,432459.701750,739287.0,550074.0,2.438814e+09,19464256.25,5.101582e+11
2099,2023-08-22,26123.41,8.358734e+07,505391.626186,1.728887,882874631,2943.055944,3.809369e+08,5.358841e+13,2.404616e+07,492337.168420,633631.0,420857.0,2.805355e+09,19465000.00,5.048637e+11
2100,2023-08-23,26036.91,9.725932e+07,505638.787271,1.687615,883294423,2942.408451,3.926239e+08,5.562144e+13,2.371891e+07,506433.385402,652036.0,417822.0,4.105798e+09,19466106.25,5.159686e+11


In [ ]:
# Check nan value
all_data[all_data.isnull().T.any()]

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap


In [ ]:
# Check duplicated rows
len(all_data['timestamp'].unique())

2102

In [ ]:
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07,15029575.00,6.474140e+09
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07,15035125.00,6.499685e+09
2,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07,15040650.00,6.458253e+09
3,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08,15046150.00,6.489359e+09
4,2016-01-06,432.43,1.677504e+06,55036.950659,0.662380,101934175,1220.496454,7.022931e+05,1.038803e+11,1.702648e+06,14113.278937,312004.0,172090.0,1.007001e+08,15051750.00,6.472554e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2097,2023-08-20,26090.70,5.461775e+07,504843.458513,1.721834,881712238,3780.623457,4.219095e+08,5.239118e+13,2.719019e+07,395205.179498,691316.0,612461.0,1.116745e+09,19463150.00,5.090198e+11
2098,2023-08-21,26189.99,3.576662e+07,505121.891014,1.633983,882324258,3333.781818,4.297226e+08,5.239118e+13,2.734033e+07,432459.701750,739287.0,550074.0,2.438814e+09,19464256.25,5.101582e+11
2099,2023-08-22,26123.41,8.358734e+07,505391.626186,1.728887,882874631,2943.055944,3.809369e+08,5.358841e+13,2.404616e+07,492337.168420,633631.0,420857.0,2.805355e+09,19465000.00,5.048637e+11
2100,2023-08-23,26036.91,9.725932e+07,505638.787271,1.687615,883294423,2942.408451,3.926239e+08,5.562144e+13,2.371891e+07,506433.385402,652036.0,417822.0,4.105798e+09,19466106.25,5.159686e+11


In [ ]:
new_columns = ['timestamp', 'market-price', 'total-bitcoins', 'market-cap'] + [col for col in all_data.columns if col not in ['timestamp', 'market-price', 'total-bitcoins', 'market-cap']]
all_data = all_data.reindex(columns=new_columns)
all_data

,timestamp,market-price,total-bitcoins,market-cap,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2016-01-01,430.89,15029575.00,6.474140e+09,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07
1,2016-01-02,434.75,15035125.00,6.499685e+09,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07
2,2016-01-04,430.78,15040650.00,6.458253e+09,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07
3,2016-01-05,434.17,15046150.00,6.489359e+09,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08
4,2016-01-06,432.43,15051750.00,6.472554e+09,1.677504e+06,55036.950659,0.662380,101934175,1220.496454,7.022931e+05,1.038803e+11,1.702648e+06,14113.278937,312004.0,172090.0,1.007001e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2097,2023-08-20,26090.70,19463150.00,5.090198e+11,5.461775e+07,504843.458513,1.721834,881712238,3780.623457,4.219095e+08,5.239118e+13,2.719019e+07,395205.179498,691316.0,612461.0,1.116745e+09
2098,2023-08-21,26189.99,19464256.25,5.101582e+11,3.576662e+07,505121.891014,1.633983,882324258,3333.781818,4.297226e+08,5.239118e+13,2.734033e+07,432459.701750,739287.0,550074.0,2.438814e+09
2099,2023-08-22,26123.41,19465000.00,5.048637e+11,8.358734e+07,505391.626186,1.728887,882874631,2943.055944,3.809369e+08,5.358841e+13,2.404616e+07,492337.168420,633631.0,420857.0,2.805355e+09
2100,2023-08-23,26036.91,19466106.25,5.159686e+11,9.725932e+07,505638.787271,1.687615,883294423,2942.408451,3.926239e+08,5.562144e+13,2.371891e+07,506433.385402,652036.0,417822.0,4.105798e+09


Once we have the daily dataset we will go to sample it at a frequency of 1 minute (1T) using the resample method. This means that the data will be organized in 1-minute time-frame, and an interpolation method will be used to fill in any missing data or holes in the dataset by estimating missing values based on the surrounding known values.

In [ ]:
# Upsampling to 1min by interpolate
all_data.set_index('timestamp', inplace=True)
all_data_30m = all_data.resample('30T').interpolate()
all_data_30m

,market-price,total-bitcoins,market-cap,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
timestamp,,,,,,,,,,,,,,,
2016-01-01 00:00:00,430.890000,1.502958e+07,6.474140e+09,2.860153e+06,54604.791735,0.493407,1.011557e+08,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.000000,124092.000000,5.834626e+07
2016-01-01 00:30:00,430.970417,1.502969e+07,6.474672e+09,2.834859e+06,54606.164986,0.495206,1.011583e+08,921.463506,6.973443e+05,1.038803e+11,1.557199e+06,8887.551433,264578.937500,124611.708333,5.818972e+07
2016-01-01 01:00:00,431.050833,1.502981e+07,6.475204e+09,2.809565e+06,54607.538237,0.497004,1.011608e+08,923.727011,6.975595e+05,1.038803e+11,1.559629e+06,8992.039015,266036.875000,125131.416667,5.803318e+07
2016-01-01 01:30:00,431.131250,1.502992e+07,6.475736e+09,2.784271e+06,54608.911488,0.498803,1.011634e+08,925.990517,6.977747e+05,1.038803e+11,1.562059e+06,9096.526597,267494.812500,125651.125000,5.787664e+07
2016-01-01 02:00:00,431.211667,1.503004e+07,6.476268e+09,2.758977e+06,54610.284739,0.500602,1.011659e+08,928.254023,6.979898e+05,1.038803e+11,1.564490e+06,9201.014178,268952.750000,126170.833333,5.772010e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-23 22:00:00,26394.254167,1.946678e+07,5.074033e+11,1.435718e+08,505858.751384,1.580342,8.836774e+08,2877.491472,3.850203e+08,5.562144e+13,2.336540e+07,463225.713145,659429.833333,400706.916667,2.462462e+09
2023-08-23 22:30:00,26402.375625,1.946680e+07,5.072086e+11,1.446244e+08,505863.750569,1.577903,8.836861e+08,2876.016086,3.848475e+08,5.562144e+13,2.335737e+07,462243.720593,659597.875000,400317.937500,2.425114e+09
2023-08-23 23:00:00,26410.497083,1.946681e+07,5.070139e+11,1.456769e+08,505868.749753,1.575465,8.836948e+08,2874.540700,3.846746e+08,5.562144e+13,2.334934e+07,461261.728042,659765.916667,399928.958333,2.387765e+09


## Output

In this last section we are going to save the dataset we just created to the Google Drive.

In [ ]:
# Output the 1h data
all_data_30m.to_parquet(GDRIVE_DATASET_RAW)